# 이미지 분석 데이터 검색

이 노트북에서는 이미지 분석이 포함된 데이터를 키워드, 벡터, 하이브리드 검색으로 활용합니다.

## 0. 필요한 라이브러리 설치

In [ ]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv azure-identity azure-search-documents openai

## 1. 환경 변수 설정

In [ ]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# Azure AI Search
AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Azure OpenAI
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-02-01")

print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"Embedding Model: {AZURE_OPENAI_EMBEDDING_DEPLOYMENT}")

## 2. 클라이언트 초기화

In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from IPython.display import display, HTML

##############################################
# 2-1. Azure 인증 및 클라이언트 생성
##############################################
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE_ENDPOINT,
    index_name=AZURE_SEARCH_INDEX_NAME,
    credential=credential
)

openai_client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

##############################################
# 2-2. 임베딩 생성 함수
##############################################
def get_embedding(text):
    response = openai_client.embeddings.create(
        input=text,
        model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT
    )
    return response.data[0].embedding

print("클라이언트 초기화 완료")

## 3. 결과 표시 헬퍼 함수

In [ ]:
##############################################
# 3. 결과 표시 헬퍼 함수
##############################################
def display_search_results(results, title="검색 결과"):
    """검색 결과를 보기 좋게 표시합니다."""
    print(f"\n{'='*80}")
    print(f"[{title}]")
    print(f"{'='*80}\n")
    
    for idx, result in enumerate(results, 1):
        score = result.get('@search.score', 'N/A')
        reranker_score = result.get('@search.reranker_score')
        
        print(f"{idx}. {result['name']} ({result['brand']})")
        print(f"   가격: {result['price']:,.0f}원")
        print(f"   검색 점수: {score}")
        
        if reranker_score:
            print(f"   Semantic Re-ranker Score: {reranker_score}")
        
        # 이미지 캡션 표시
        if result.get('imageCaption'):
            print(f"   💬 이미지 캡션: {result['imageCaption']}")
        
        # 이미지 태그 표시
        if result.get('imageTags'):
            tags = ', '.join(result['imageTags'][:5])
            print(f"   🏷️  태그: {tags}")
        
        # 제품 설명
        print(f"   📝 설명: {result['description'][:100]}...")
        
        # 이미지 표시
        if result.get('imageUrl'):
            display(HTML(f'<img src="{result["imageUrl"]}" width="300" style="margin: 10px 0;" />'))
        
        print()

print("헬퍼 함수 정의 완료")

## 4. 이미지 정보 기반 검색

검색 범위: imageCaption, imageDescription, imageTags

In [ ]:
##############################################
# 4. 이미지 캡션 기반 검색
##############################################
search_query = "검정색 가방"

results = search_client.search(
    search_text=search_query,
    top=3,
    search_fields=["imageCaption", "imageDescription", "imageTags"],  # 이미지 캡션과 설명만 검색
    select=["id", "name", "brand", "price", "description", "imageUrl", "imageCaption", "imageDescription", "imageTags"]
)

print(f"🔍 검색어: '{search_query}'")
print(f"💬 검색 필드: imageCaption, imageDescription (이미지 분석 결과만)")

display_search_results(results, "이미지 캡션/설명 검색 결과")

## 5. 이미지 정보를 포함한 하이브리드 검색

검색 범위: name, description, imageCaption, imageDescription, imageTags + 벡터 임베딩

In [ ]:
from azure.search.documents.models import VectorizedQuery

##############################################
# 5. 하이브리드 검색
##############################################
search_query = "검정색 가죽 가방 찾아줘"

# 검색어를 벡터로 변환
query_vector = get_embedding(search_query)

vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    top=3,
    search_fields=["name", "description", "imageCaption", "imageDescription", "imageTags"],
    select=["id", "name", "brand", "price", "description", "imageUrl", "imageCaption", "imageTags"]
)

print(f"🔍 검색어: '{search_query}'")
print(f"🔗 검색 방식: 하이브리드 (키워드 + 벡터)")

display_search_results(results, "하이브리드 검색 결과")

## 6. 복합 검색 예시

검색 범위: name, description, brand, imageCaption, imageDescription, imageTags + 벡터 임베딩

In [ ]:
##############################################
# 9. 복합 검색 - 제품 설명과 이미지 정보 모두 활용
##############################################
search_query = "스타일리시한 도시용 백팩"

# 벡터 검색 추가
query_vector = get_embedding(search_query)
vector_query = VectorizedQuery(
    vector=query_vector,
    k_nearest_neighbors=5,
    fields="descriptionVector"
)

results = search_client.search(
    search_text=search_query,
    vector_queries=[vector_query],
    top=3,
    search_fields=["name", "description", "brand", "imageCaption", "imageDescription", "imageTags"],
    select=["id", "name", "brand", "price", "description", "imageUrl", "imageCaption", "imageDescription", "imageTags"]
)

print(f"🔍 검색어: '{search_query}'")
print(f"📚 검색 필드: 모든 텍스트 필드 (제품 정보 + 이미지 분석 결과)")
print(f"🔗 검색 방식: 하이브리드 (키워드 + 벡터)")

print(f"\n{'='*80}")
print(f"[복합 검색 결과]")
print(f"{'='*80}\n")

for idx, result in enumerate(results, 1):
    score = result.get('@search.score', 'N/A')
    
    print(f"{idx}. {result['name']} ({result['brand']}) - {result['price']:,.0f}원")
    print(f"   검색 점수: {score}")
    
    print(f"\n   📝 제품 설명:")
    print(f"      {result['description']}")
    
    if result.get('imageCaption'):
        print(f"\n   💬 이미지 캡션:")
        print(f"      {result['imageCaption']}")
    
    if result.get('imageDescription'):
        print(f"\n   🖼️  이미지 상세 설명:")
        print(f"      {result['imageDescription'][:150]}...")
    
    if result.get('imageTags'):
        print(f"\n   🏷️  이미지 태그:")
        print(f"      {', '.join(result['imageTags'])}")
    
    if result.get('imageUrl'):
        display(HTML(f'<img src="{result["imageUrl"]}" width="300" style="margin: 10px 0; border-radius: 8px;" />'))
    
    print("\n" + "-"*80 + "\n")